# Machine learning su aplicación en Biología

__Análisis de datos biológicos utilizando métodos de machine learning__

_MeIA_

`2023`

## Support Vector Machines

Lás máquinas de vectores de soporte son clasificadores lineales, porque establecen límites de decisión lineales. Tienen como objetivo encontrar un hiperplano o límite de decisión que se ajuste mejor y separe el espacio n-dimensional en clases o grupos separados. Este hiperplano es el límite de decisión óptimo.

Para aplicar SVM, los datos deben ser clasificables linealmente, lo que significa que debe existir una línea o un plano desde donde se puedan separar los grupos o clases.

<img src="../imagenes/svm.png" width="500" height="400"/>
En a) podemos ver que el dataset es linealmente separable, sin embargo puede haber muchas líneas que pueden separar los datos en diferentes clases. SVM selecciona los puntos óptimos que ayudan a crear el hiperplano. Estos puntos se denominan vectores de soporte, que permiten identificar el límite de decisión.

#### Truco del Kernel
Muchas veces los datos del mundo real no tiene datos linealmente separables. Hasta cierto punto el uso de un margen suave puede ayudar a clasificar los datos de forma lineal, permitiendo algunos errores de clasificación. 

Pero para datasets que no tienen ningún límite lineal único, se aplica el truco de Kernel. Este tiene como objetivo mapear los datos de baja dimensión a una dimensión más alta donde es separable linealmente. 
<img src="../imagenes/svm1.png" width="500" height="400"/>
En a) tenemos un dataset unidimensional que tiene dos clase, amarillo y azul. No hay forma de que un punto pueda diferenciar estas dos clases. La solución es adicionar una característica adicional a los datos para que podamos diferenciar las dos clases usando una linea en el espacio bidimensional.



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#vamos a utilizar los datos de enfermedad cardiáca 
dataset = pd.read_csv('../Datos/heart.csv')
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


### Dividimos el dataset en un conjunto de entrenamiento y prueba

In [3]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### Entrenamiento del modelo de SVM

In [4]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)

SVC()

### Predicción y evaluación

In [5]:
#Utilizaremos los valores por default
preds = model.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,preds))
print('\n')
print(classification_report(y_test,preds))

[[23 16]
 [ 6 46]]


              precision    recall  f1-score   support

           0       0.79      0.59      0.68        39
           1       0.74      0.88      0.81        52

    accuracy                           0.76        91
   macro avg       0.77      0.74      0.74        91
weighted avg       0.76      0.76      0.75        91



Con la configuración por default, obtenemos un exactitud del 69% que es mejor que la precisión de referencia.

Vamos a intentar mejorar la predicción utilizando el valor c, que brinda flexibilidad a los margenes, lo que nos permite clasificar algunos elementos de manera erronea. De la misma manera, la elección del valor "gamma", que define el efecto de las nuevas carácteristicas mientras aumenta las dimensiones  utilizando una función de base radial, dependiendo de cada caso. Si bien la función de base radial es la función kernel más utilizada, también existen otras funciones, como polinomio o sigmoidal. 

Por lo tanto el elegir el valor c, gamma y la función kernel correcta son tareas desafiantes. 

Utilizaremos la función "GridSearchCV", que realiza una búsqueda de cuadrícula que nos permitirá identificar los parámetros.



In [6]:
from sklearn.model_selection import GridSearchCV
#Primero vamos a crear un diccionario de parámetros, 
#donde proporcionaremos varios valores de c y gamma

#Una búsqueda de cuadrícula requiere una cantidad significativa de tiempo 
#por esto solo utilizaremos "radial basis function (RBF)" como la única función del kernel.
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001],
'kernel': ['rbf']}
grid = GridSearchCV(SVC(),param_grid,verbose=3)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.535 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.535 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.548 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.535 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.535 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.548 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.535 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

Una vez completa la búsqueda de cuadrícula, podemos recuperar los mejores parámetros utilizando el atributo "best_params_" 
Obtenemos c=100, gamma=0.0001

In [7]:
grid.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Podemos utilizar 'GridSearchCV'para predecir , ya que tendrá todas las propiedades del clasificador con los mejores parámetros.

In [8]:
grid_predictions = grid.predict(X_test)
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))

[[24 15]
 [11 41]]


              precision    recall  f1-score   support

           0       0.69      0.62      0.65        39
           1       0.73      0.79      0.76        52

    accuracy                           0.71        91
   macro avg       0.71      0.70      0.70        91
weighted avg       0.71      0.71      0.71        91



El modelo ajustado ha aumentado de manera convincente la exactitud. SVM son bastante sensibles a sus parámetros, por lo que es extremadamente importante encontrar parámetros óptimos para cada proyecto SVM.

### Predicción de especies de trigo basada en datos de semillas de trigo

Utilizaremos el dataset de semillas de trigo, de tres variedades diferentes: Rama, Rosa y Canadian, se examinarón 70 elementos de cada variedad, seleccionadas al azar. Se detectó una visualización de alta calidad de la estructura interna del n{ucleo utilizando una técnica de rayos x. Las imágenes se registaron en plxas KODAK de rayos x de 13x18cm.

Los granos de trigo fueron cosechados desde campos experimentales de Instituto de Agrofísica de la Academia de Ciencias de Polonia.

1. Área
2. Perímetro
3. Espacio compacto
4. Longitud del núcleo
5. Ancho del núcleo
6. Coeficiente de asimetría
7. Longitud del surco del grano

In [9]:
dataset = pd.read_csv('../Datos/seeds_dataset.csv')
dataset.head()

,ID,area,perimeter,compactness,lengthOfKernel,widthOfKernel,asymmetryCoefficient,lengthOfKernelGroove,seedType
0,1,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,2,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,3,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,4,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,5,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


Este consite de medidas de tres semillas de trigo: Kama, Rosa y Canadian.

Estas tres especies estan igualmente distribuidas, por lo que podemos decir que nuestra exactitud base es de 33%.


In [10]:
dataset['seedType'].value_counts()

1    70
2    70
3    70
Name: seedType, dtype: int64

In [11]:
#Verificamos la existencia de valores null
dataset.isnull().sum()

ID                      0
area                    0
perimeter               0
compactness             0
lengthOfKernel          0
widthOfKernel           0
asymmetryCoefficient    0
lengthOfKernelGroove    0
seedType                0
dtype: int64

#### Dividimos nuestro dataset

In [12]:
X = dataset.iloc[:,1:-1]
y = dataset.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,
random_state = 0)

#### Entrenamos el modelo y ajustamos sus parámetros mediante la busqueda de GridSearch

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma':
[1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.844 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.844 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.871 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.774 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.742 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.875 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.906 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.935 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.968 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.903 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.781 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [14]:
#verificamos los mejores parametros
grid.best_params_

{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}

In [15]:
grid_predictions = grid.predict(X_test)
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           1       0.89      1.00      0.94        17
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00        15

    accuracy                           0.96        53
   macro avg       0.96      0.97      0.96        53
weighted avg       0.97      0.96      0.96        53



Obtuvimos una exactitud del 96%.